In [59]:
import mysql.connector
import pandas as pd

In [7]:
df_staging = pd.read_csv('staging_table_202403151057.csv')

In [62]:
province_table = df_staging[['kode_prov', 'nama_prov']]
province_table = province_table.drop_duplicates()
province_table.rename(columns={'kode_prov':'province_id', 'nama_prov':'province_name'}, inplace=True)

In [76]:
province_table

,province_id,province_name
0,32,Jawa Barat


In [68]:
district_table = df_staging[['kode_kab', 'kode_prov', 'nama_kab', 'nama_prov']]
district_table = district_table.drop_duplicates()
district_table.rename(columns={'kode_kab':'district_id', 'kode_prov':'province_id', 'nama_kab':'district_name', 'nama_prov':'province_name'}, inplace=True)

In [80]:
district_table = district_table.drop(['province_name'], axis =1)

In [81]:
district_table

,district_id,province_id,district_name
0,3204,32,Kabupaten Bandung
1,3217,32,Kabupaten Bandung Barat
2,3216,32,Kabupaten Bekasi
3,3201,32,Kabupaten Bogor
4,3207,32,Kabupaten Ciamis
5,3203,32,Kabupaten Cianjur
6,3209,32,Kabupaten Cirebon
7,3205,32,Kabupaten Garut
8,3212,32,Kabupaten Indramayu
9,3215,32,Kabupaten Karawang


In [124]:
df_staging.columns

Index(['CLOSECONTACT', 'CONFIRMATION', 'PROBABLE', 'SUSPECT',
       'closecontact_dikarantina', 'closecontact_discarded',
       'closecontact_meninggal', 'confirmation_meninggal',
       'confirmation_sembuh', 'kode_kab', 'kode_prov', 'nama_kab', 'nama_prov',
       'probable_diisolasi', 'probable_discarded', 'probable_meninggal',
       'suspect_diisolasi', 'suspect_discarded', 'suspect_meninggal',
       'tanggal'],
      dtype='object')

In [127]:
case_table_dict = {'status_name': ['CLOSECONTACT','CLOSECONTACT','CLOSECONTACT',
                              'CONFIRMATION','CONFIRMATION',
                              'PROBABLE','PROBABLE','PROBABLE',
                              'SUSPECT', 'SUSPECT', 'SUSPECT'],
                'status_detail': ['closecontact_dikarantina', 'closecontact_discarded', 'closecontact_meninggal', 
                                'confirmation_meninggal', 'confirmation_sembuh', 
                                'probable_diisolasi', 'probable_discarded', 'probable_meninggal', 
                                'suspect_diisolasi', 'suspect_discarded', 'suspect_meninggal']
            }

In [128]:
case_table = pd.DataFrame(case_table_dict)

In [ ]:
case_table = case_table.reset_index().rename(columns={'index':'Id'})

In [133]:
case_table

,Id,status_name,status_detail
0,0,CLOSECONTACT,closecontact_dikarantina
1,1,CLOSECONTACT,closecontact_discarded
2,2,CLOSECONTACT,closecontact_meninggal
3,3,CONFIRMATION,confirmation_meninggal
4,4,CONFIRMATION,confirmation_sembuh
5,5,PROBABLE,probable_diisolasi
6,6,PROBABLE,probable_discarded
7,7,PROBABLE,probable_meninggal
8,8,SUSPECT,suspect_diisolasi
9,9,SUSPECT,suspect_discarded


In [72]:
# Group by province and calculate the total cases for each status
province_agg = df_staging.groupby('nama_prov').agg({
    'CLOSECONTACT': 'sum',
    'CONFIRMATION': 'sum',
    'PROBABLE': 'sum',
    'SUSPECT': 'sum'
}).reset_index()

# Group by province and district, then calculate the total cases for each status
district_agg = df_staging.groupby(['nama_prov', 'nama_kab']).agg({
    'CLOSECONTACT': 'sum',
    'CONFIRMATION': 'sum',
    'PROBABLE': 'sum',
    'SUSPECT': 'sum'
}).reset_index()

# # Merge province and district aggregation data with the respective dimension tables
province_agg = pd.merge(province_agg, province_table, left_on='nama_prov', right_on='province_name', how='left')
district_agg = pd.merge(district_agg, district_table, left_on=['nama_prov', 'nama_kab'], right_on=['province_name', 'district_name'], how='left')

# # Rename columns to match dimension table
province_agg.rename(columns={'province_id': 'province_id', 'nama_prov': 'province_name'}, inplace=True)
district_agg.rename(columns={'district_id': 'district_id', 'nama_prov': 'province_name', 'nama_kab': 'district_name'}, inplace=True)

# # Insert aggregation data into the case table
case_table = pd.concat([province_agg, district_agg], ignore_index=True)
case_table.rename(columns={'province_name': 'nama_prov', 'district_name': 'nama_kab'}, inplace=True)

# # Convert DataFrame to a list of dictionaries for easy insertion into a database
aggregation_data = case_table.to_dict(orient='records')

# # Now you can insert aggregation_data into your PostgreSQL database

TypeError: concat() got an unexpected keyword argument 'reset_index'

In [32]:
province_table = df_staging[['kode_prov', 'nama_prov']]
province_table = province_table.drop_duplicates()
province_table.rename(columns={'kode_prov':'province_id', 'nama_prov':'province_name'}, inplace=True)

In [75]:
district_agg

,province_name,district_name,CLOSECONTACT,CONFIRMATION,PROBABLE,SUSPECT,district_id,province_id,district_name,province_name
0,Jawa Barat,Kabupaten Bandung,1077,1023,468,6732,3204,32,Kabupaten Bandung,Jawa Barat
1,Jawa Barat,Kabupaten Bandung Barat,1602,348,126,2328,3217,32,Kabupaten Bandung Barat,Jawa Barat
2,Jawa Barat,Kabupaten Bekasi,6783,1479,603,16953,3216,32,Kabupaten Bekasi,Jawa Barat
3,Jawa Barat,Kabupaten Bogor,0,1407,2958,12057,3201,32,Kabupaten Bogor,Jawa Barat
4,Jawa Barat,Kabupaten Ciamis,3885,45,54,6225,3207,32,Kabupaten Ciamis,Jawa Barat
5,Jawa Barat,Kabupaten Cianjur,408,135,3,3651,3203,32,Kabupaten Cianjur,Jawa Barat
6,Jawa Barat,Kabupaten Cirebon,1359,225,18,918,3209,32,Kabupaten Cirebon,Jawa Barat
7,Jawa Barat,Kabupaten Garut,7152,159,0,8685,3205,32,Kabupaten Garut,Jawa Barat
8,Jawa Barat,Kabupaten Indramayu,813,156,1323,5010,3212,32,Kabupaten Indramayu,Jawa Barat
9,Jawa Barat,Kabupaten Karawang,4416,372,1026,18327,3215,32,Kabupaten Karawang,Jawa Barat
